# HAVEN ReID & ADL Analysis

Analyze logs from SQLite database.

In [4]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

# Connect to DB (Robust search)
cwd = Path(os.getcwd())
db_path = cwd / "database" / "haven_reid.db"

# If running from root, check backend/database
if not db_path.exists():
    db_path = cwd / "backend" / "database" / "haven_reid.db"
    
# If still not found, try hardcoded path
if not db_path.exists():
    db_path = Path(r"D:\HAVEN\backend\database\haven_reid.db")

print(f"Target DB Path: {db_path}")

if db_path.exists():
    conn = sqlite3.connect(str(db_path))
    
    # List valid sessions
    try:
        sessions = pd.read_sql("SELECT DISTINCT session_id, count(*) as count FROM event_log GROUP BY session_id ORDER BY session_id DESC", conn)
        print("Available Sessions:")
        display(sessions.head())
    except Exception as e:
        print(f"Error reading table: {e}")
else:
    print("❌ Database file not found!")

Target DB Path: d:\HAVEN\backend\database\haven_reid.db
Available Sessions:


,session_id,count


In [5]:
# Select latest session
if 'sessions' in locals() and not sessions.empty:
    latest_session = sessions['session_id'].iloc[0]
    print(f"Analyzing session: {latest_session}")

    # Load data
    df = pd.read_sql(f"SELECT * FROM event_log WHERE session_id = '{latest_session}'", conn)
    df['timestamp_dt'] = pd.to_datetime(df['created_at'])
    display(df.head())
else:
    print("No data to analyze.")

No data to analyze.


## 1. Posture Distribution

In [6]:
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=df, x='camera', hue='posture')
    plt.title("Posture Distribution by Camera")
    plt.show()

## 2. Person Tracking Timeline

In [7]:
if 'df' in locals() and not df.empty:
    plt.figure(figsize=(12, 6))
    sns.stripplot(data=df, x='frame', y='camera', hue='global_id', jitter=0.2, alpha=0.6)
    plt.title("Person Tracking across Cameras over Time")
    plt.show()

## 3. Danger Objects Detected

In [8]:
if 'df' in locals() and not df.empty:
    # Filter frames with objects
    obj_df = df[df['objects'] != ""]
    if not obj_df.empty:
        display(obj_df[['camera', 'frame', 'objects']].head(10))
        
        plt.figure(figsize=(8, 4))
        obj_df['camera'].value_counts().plot(kind='bar')
        plt.title("Danger Object Detections by Camera")
        plt.show()
    else:
        print("No danger objects detected.")